# CollabLLM Dataset Construction 

There are two types of datasets in CollabLLM:
- SingTurnDataset: Any single-turn tasks can be defined as a SingTurnDataset.
- MultiTurnDataset: Any multiturn conversation can be stored as MultiTurnDataset.

## Single-turn dataset

Demonstrates:
1. Creating a list of single-turn chat entries.
2. Wrapping it with SingTurnDataset.
3. Converting to a HuggingFace DatasetDict.
4. Inspecting splits and sample entries.

In [2]:
from pprint import pprint
from datasets import DatasetDict
from collabllm.datasets import SingleTurnDataset

# ------------------------------------------------------ #
# 1) Prepare some toy single-turn data                   #
# ------------------------------------------------------ #
toy_data = [
    {
        "prompt": "What is the capital of France?",
        "completion": "Paris.",
        "difficulty": "easy",
    },
    {
        "prompt": "Compute 15 * 7.",
        "completion": "105.",
        "difficulty": "easy",
    },
    {
        "prompt": "Explain the theory of relativity in brief.",
        "completion": "It’s a theory by Einstein explaining how space and time are linked and how massive objects curve spacetime. In short, E=mc².",
        "difficulty": "hard",
    },
    {
        "prompt": "Who wrote 'Pride and Prejudice'?",
        "completion": "Jane Austen.",
        "difficulty": "medium",
    },
    {
        "prompt": "Translate 'Hello' to Spanish.",
        "completion": "Hola.",
        "difficulty": "easy",
    },
]

# ------------------------------------------------------ #
# 2) Initialize SingleTurnDataset                          #
# ------------------------------------------------------ #
dataset = SingleTurnDataset(toy_data, eval_ratio=0.2, seed=123)

In [3]:
# ------------------------------------------------------ #
# 3) Convert to HuggingFace DatasetDict                  #
# ------------------------------------------------------ #
hf_datasets: DatasetDict = dataset.to_hf_dataset()
print("Dataset splits:", hf_datasets)

# ------------------------------------------------------ #
# 4) Inspect split sizes                                #
# ------------------------------------------------------ #
splits_info = dataset.get_splits_info()
print("Split info:", splits_info)

# ------------------------------------------------------ #
# 5) Peek at one example from each split                 #
# ------------------------------------------------------ #
for split_name, split_ds in hf_datasets.items():
    print(f"\n--- {split_name.upper()} split ---")
    # Each row has: single_turn_prompt, single_turn_completion, metadata
    row0 = split_ds[0]
    pprint(row0)

# ------------------------------------------------------ #
# 6) Accessing entries via __getitem__                   #
# ------------------------------------------------------ #
print("\nFirst entry via __getitem__:")
print(dataset[0])

print("\nTotal entries:", len(dataset))

Dataset splits: DatasetDict({
    train: Dataset({
        features: ['single_turn_prompt', 'single_turn_completion', 'single_turn_metadata'],
        num_rows: 4
    })
    eval: Dataset({
        features: ['single_turn_prompt', 'single_turn_completion', 'single_turn_metadata'],
        num_rows: 1
    })
})
Split info: {'train': 4, 'eval': 1}

--- TRAIN split ---
{'single_turn_completion': '105.',
 'single_turn_metadata': {'difficulty': 'easy'},
 'single_turn_prompt': 'Compute 15 * 7.'}

--- EVAL split ---
{'single_turn_completion': 'Jane Austen.',
 'single_turn_metadata': {'difficulty': 'medium'},
 'single_turn_prompt': "Who wrote 'Pride and Prejudice'?"}

First entry via __getitem__:
{'prompt': 'What is the capital of France?', 'completion': 'Paris.', 'difficulty': 'easy'}

Total entries: 5


## Multiturn dataset

There are two ways to create a multiturn dataset:
- Provide a list of data entries (by separated rows or nested dictionary).
- Specifify a huggingface dataset repo name ([example format](https://huggingface.co/datasets/collabllm/collabllm-multiturn-math-hard)) or a local directory containing a huggingface dataset.

### Method 1.1: Create a multiturn dataset from a list of data entries (separated rows)

In [4]:
from pprint import pprint
from datasets import Dataset, DatasetDict
from collabllm.datasets import MultiturnDataset

# ------------------------------------------------------ #
# 1) Toy corpus: three conversations, four rows total    #
# ------------------------------------------------------ #
toy_data = [
    {  # convo A, assistant turn 1
        "prompt": [
            {"role": "user", "content": "Hi – tell me a joke."},
        ],
        "completion": "Why did the chicken cross the road? To get to the other side!",
        "conv_id": 1,
        "score": 0.7,
        "single_turn_prompt": "Tell me a joke.",
        "single_turn_completion": "Why did the chicken cross the road?",
        "single_turn_metadata": {"topic": "jokes"},
    },
    {  # convo A, *same* turn, alternative completion (lower score)
        "prompt": [
            {"role": "user", "content": "Hi – tell me a joke."},
        ],
        "completion": "I don't know any jokes.",
        "conv_id": 1,
        "score": 0.1,
        "single_turn_prompt": "Tell me a joke.",
        "single_turn_completion": "I don't know any jokes.",
        "single_turn_metadata": {"topic": "jokes"},
    },
    {  # convo A, *same* turn, alternative completion (lower score)
        "prompt": [
            {"role": "user", "content": "Hi – tell me a joke."},
            {"role": "assistant", "content": "Why did the chicken cross the road? To get to the other side!"},
            {"role": "user", "content": "And another one?"},
        ],
        "completion": "What do you call a bear with no teeth? A gummy bear!",
        "conv_id": 1,
        "score": 0.8,
        "single_turn_prompt": "Tell me a joke.",
        "single_turn_completion": "I don't know any jokes.",
        "single_turn_metadata": {"topic": "jokes"},
    },
    {  # convo A, *same* turn, alternative completion (lower score)
        "prompt": [
            {"role": "user", "content": "Hi – tell me a joke."},
            {"role": "assistant", "content": "Why did the chicken cross the road? To get to the other side!"},
            {"role": "user", "content": "And another one?"},
        ],
        "completion": "I don't know any jokes.",
        "conv_id": 1,
        "score": 0.1,
        "single_turn_prompt": "Tell me a joke.",
        "single_turn_completion": "I don't know any jokes.",
        "single_turn_metadata": {"topic": "jokes"},
    },
    {  # convo B, assistant turn 2
        "prompt": [
            {"role": "user", "content": "Sum 2+2"},
            {"role": "assistant", "content": "4"},
            {"role": "user", "content": "And 3+3?"},
        ],
        "completion": "6",
        "conv_id": 2,
        "score": 0.9,
        "single_turn_prompt": "What is 3+3?",
        "single_turn_completion": "6",
        "single_turn_metadata": {"topic": "math"},
    },
    {  # convo C, single message
        "prompt": [{"role": "user", "content": "Quote Shakespeare"}],
        "completion": "To be, or not to be.",
        "conv_id": 2,
        "score": 0.5,
        "single_turn_prompt": "Quote Shakespeare",
        "single_turn_completion": "To be, or not to be.",
        "single_turn_metadata": {"topic": "literature"},
    },
]

ds = MultiturnDataset(toy_data, seed=42, add_system_prompt=False)

# Add system prompt (by default)
# ds = MultiturnDataset(toy_data, seed=42, add_system_prompt=True)

# ------------------------------------------------------ #
# 2) SFT                                                 #
# ------------------------------------------------------ #
sft = ds.to_sft_dataset()
print("\nSFT:")
print(sft)
pprint(sft["train"][0])

# ------------------------------------------------------ #
# 3) DPO (gap filter 0.2)                                #
# ------------------------------------------------------ #
dpo = ds.to_dpo_dataset(minimum_gap=0.2)
print("\nDPO:")
print(dpo)
if len(dpo["train"]) > 0:
    pprint(dpo["train"][0])

# ------------------------------------------------------ #
# 4) Inputs                                              #
# ------------------------------------------------------ #
inputs = ds.to_inputs_dataset()
print("\nInputs:")
print(inputs)
pprint(inputs["train"][0])


2025-06-08 00:41:46,510 [INFO] collabllm.datasets.multiturn: Converted 2 pairs (minimum_gap=0.2, ratio=0.33)



SFT:
DatasetDict({
    train: Dataset({
        features: ['messages'],
        num_rows: 2
    })
    eval: Dataset({
        features: ['messages'],
        num_rows: 0
    })
})
{'messages': [{'content': 'Hi – tell me a joke.', 'role': 'user'},
              {'content': 'Why did the chicken cross the road? To get to the '
                          'other side!',
               'role': 'assistant'},
              {'content': 'And another one?', 'role': 'user'},
              {'content': 'What do you call a bear with no teeth? A gummy '
                          'bear!',
               'role': 'assistant'}]}

DPO:
DatasetDict({
    train: Dataset({
        features: ['prompt', 'chosen', 'rejected', 'score_chosen', 'score_rejected'],
        num_rows: 2
    })
    eval: Dataset({
        features: ['prompt', 'chosen', 'rejected', 'score_chosen', 'score_rejected'],
        num_rows: 0
    })
})
{'chosen': 'Why did the chicken cross the road? To get to the other side!',
 'prompt': [{'co

### Method 1.2: Create a multiturn dataset from a list of data entries (nested dictionary)

In [5]:
from pprint import pprint

from datasets import DatasetDict
from collabllm.datasets import MultiturnDataset

# ------------------------------------------------------ #
# 1) Example with nested input format                    #
# ------------------------------------------------------ #
nested_data = [
    {
        "single_turn_prompt": "Tell me a joke.",
        "single_turn_completion": "Why did the chicken cross the road?",
        "single_turn_metadata": {"category": "humor"},
        "turns": [
            {
                "prompt": [{"role": "user", "content": "Hi, tell me a joke."}],
                "responses": [
                    {"completion": "Why did the chicken cross the road? To get to the other side!", "score": 0.7},
                    {"completion": "I don’t know any jokes.", "score": 0.2},
                ],
            },
            {
                "prompt": [
                    {"role": "user", "content": "Another one?"},
                    {"role": "assistant", "content": "Why did the chicken cross the road? To get to the other side!"},
                    {"role": "user", "content": "Yes, another."},
                ],
                "responses": [
                    {"completion": "What do you call a bear with no teeth? A gummy bear!", "score": 0.8},
                ],
            },
        ],
    },
    {
        "single_turn_prompt": "Sum 2+2.",
        "single_turn_completion": "2+2=4",
        "single_turn_metadata": {"category": "math"},
        "turns": [
            {
                "prompt": [{"role": "user", "content": "Sum 2+2"}],
                "responses": [
                    {"completion": "4", "score": 0.9},
                ],
            }
        ],
    },
]

ds_nested = MultiturnDataset(nested_data, seed=123, add_system_prompt=False)

# Add system prompt (by default)
# ds_nested = MultiturnDataset(nested_data, seed=123, add_system_prompt=True)

print("=== SFT from nested data ===")
sft_ds: DatasetDict = ds_nested.to_sft_dataset()
print(sft_ds)
pprint(sft_ds["train"][0])  # one example from train split

print("\n=== DPO from nested data ===")
dpo_ds: DatasetDict = ds_nested.to_dpo_dataset(minimum_gap=0.3)
print(dpo_ds)
if len(dpo_ds["train"]) > 0:
    pprint(dpo_ds["train"][0])

print("\n=== Inputs from nested data ===")
inputs_ds: DatasetDict = ds_nested.to_inputs_dataset()
print(inputs_ds)
pprint(inputs_ds["train"][0])

2025-06-08 00:41:46,607 [INFO] collabllm.datasets.multiturn: Converted 1 pairs (minimum_gap=0.3, ratio=0.25)


=== SFT from nested data ===
DatasetDict({
    train: Dataset({
        features: ['messages'],
        num_rows: 2
    })
    eval: Dataset({
        features: ['messages'],
        num_rows: 0
    })
})
{'messages': [{'content': 'Another one?', 'role': 'user'},
              {'content': 'Why did the chicken cross the road? To get to the '
                          'other side!',
               'role': 'assistant'},
              {'content': 'Yes, another.', 'role': 'user'},
              {'content': 'What do you call a bear with no teeth? A gummy '
                          'bear!',
               'role': 'assistant'}]}

=== DPO from nested data ===
DatasetDict({
    train: Dataset({
        features: ['prompt', 'chosen', 'rejected', 'score_chosen', 'score_rejected'],
        num_rows: 1
    })
    eval: Dataset({
        features: ['prompt', 'chosen', 'rejected', 'score_chosen', 'score_rejected'],
        num_rows: 0
    })
})
{'chosen': 'Why did the chicken cross the road? To get t

### Method 2.1: Create a multiturn dataset from huggingface dataset repo

In [6]:
from collabllm.datasets import MultiturnDataset

ds = MultiturnDataset('collabllm/collabllm-multiturn-math-hard', add_system_prompt=True)

print("=== SFT ===")
sft_ds: DatasetDict = ds.to_sft_dataset(eval_ratio=0.1)
print(sft_ds)
print(sft_ds["train"][0])  # one example from train split

=== SFT ===
DatasetDict({
    train: Dataset({
        features: ['messages'],
        num_rows: 450
    })
    eval: Dataset({
        features: ['messages'],
        num_rows: 50
    })
})
{'messages': [{'content': 'The assistant is designed to be helpful, proactive, and highly interactive.\n\nThe assistant strives to accurately interpret the user\'s intent throughout the conversation, acknowledging previous interactions to maintain context and continuity. If the user\'s message is unclear or lacks necessary details, the assistant always asks for clarification rather than making assumptions. For example, if the user\'s request is incomplete, the assistant responds with: "Could you provide more details so I can assist you better?"\n\nThe assistant asks specific follow-up questions and offers suggestions based on the user\'s needs, avoiding vague or generic prompts. It proactively provides guidance and potential next steps, especially in complex tasks such as writing, analysis, coding,

When specify DPO dataset, set `minimum_gap` to filter out pairs where the score difference is below `minimum_gap`

In [7]:
print("\n=== DPO (minimum gap = 0.1) ===")
dpo_ds: DatasetDict = ds.to_dpo_dataset(minimum_gap=0.1, eval_ratio=0.1)
print(dpo_ds)


print("\n=== DPO (minimum gap = 0.2) ===")
dpo_ds: DatasetDict = ds.to_dpo_dataset(minimum_gap=0.2, eval_ratio=0.1)
print(dpo_ds)
if len(dpo_ds["train"]) > 0:
    print(dpo_ds["train"][0])

2025-06-08 00:41:55,579 [INFO] collabllm.datasets.multiturn: Converted 1759 pairs (minimum_gap=0.1, ratio=0.25)


2025-06-08 00:41:55,662 [INFO] collabllm.datasets.multiturn: Converted 1237 pairs (minimum_gap=0.2, ratio=0.18)



=== DPO (minimum gap = 0.1) ===
DatasetDict({
    train: Dataset({
        features: ['prompt', 'chosen', 'rejected', 'score_chosen', 'score_rejected'],
        num_rows: 1584
    })
    eval: Dataset({
        features: ['prompt', 'chosen', 'rejected', 'score_chosen', 'score_rejected'],
        num_rows: 175
    })
})

=== DPO (minimum gap = 0.2) ===
DatasetDict({
    train: Dataset({
        features: ['prompt', 'chosen', 'rejected', 'score_chosen', 'score_rejected'],
        num_rows: 1114
    })
    eval: Dataset({
        features: ['prompt', 'chosen', 'rejected', 'score_chosen', 'score_rejected'],
        num_rows: 123
    })
})
{'prompt': [{'content': 'The assistant is designed to be helpful, proactive, and highly interactive.\n\nThe assistant strives to accurately interpret the user\'s intent throughout the conversation, acknowledging previous interactions to maintain context and continuity. If the user\'s message is unclear or lacks necessary details, the assistant always ask

In [8]:
print("\n=== Inputs ===")
inputs_ds: DatasetDict = ds.to_inputs_dataset(eval_ratio=0.1)
print(inputs_ds)
print(inputs_ds["train"][0])


=== Inputs ===
DatasetDict({
    train: Dataset({
        features: ['prompt', 'single_turn_prompt', 'single_turn_completion', 'single_turn_metadata'],
        num_rows: 3150
    })
    eval: Dataset({
        features: ['prompt', 'single_turn_prompt', 'single_turn_completion', 'single_turn_metadata'],
        num_rows: 350
    })
})
{'prompt': [{'content': 'The assistant is designed to be helpful, proactive, and highly interactive.\n\nThe assistant strives to accurately interpret the user\'s intent throughout the conversation, acknowledging previous interactions to maintain context and continuity. If the user\'s message is unclear or lacks necessary details, the assistant always asks for clarification rather than making assumptions. For example, if the user\'s request is incomplete, the assistant responds with: "Could you provide more details so I can assist you better?"\n\nThe assistant asks specific follow-up questions and offers suggestions based on the user\'s needs, avoiding vag

### Method 2.2: Create a multiturn dataset from a local directory

You can load the dataset from a local directory which is saved by `ds.save_to_disk` where ds is a HuggingFace Dataset object.